In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from sklearn.metrics import classification_report, accuracy_score
from tabulate import tabulate  # For formatting tables

In [ ]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Functions
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [ ]:
# Custom Dataset Class
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
            
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [ ]:
# Load training and validation data
dec_data = np.loadtxt('Train_Dst_NoAuction_DecPre_CF_7.txt')
dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

# Load test data
dec_test1 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_7.txt')
dec_test2 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_8.txt')
dec_test3 = np.loadtxt('Test_Dst_NoAuction_DecPre_CF_9.txt')
dec_test = np.hstack((dec_test1, dec_test2, dec_test3))

# Print data shapes
print(dec_train.shape, dec_val.shape, dec_test.shape)

In [ ]:
# Creating Dataset Instances and Data Loaders

batch_size = 64
dataset_train = Dataset(data=dec_train, k=4, num_classes=3, T=100)
dataset_val = Dataset(data=dec_val, k=4, num_classes=3, T=100)
dataset_test = Dataset(data=dec_test, k=4, num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)

In [ ]:
# Viewing dataset_train
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

In [ ]:
# List of model names
model_names = ['CNN1', 'DeepLOB', 'BinCTABL', 'AxialLOB']

# Initialize dictionaries to store file paths
loss_files = {
    'train': {},
    'val': {}
}

best_model_files = {}

# Assuming all CSV and model files are in the current working directory
for model in model_names:
    loss_files['train'][model] = f'train_losses_{model}.csv'
    loss_files['val'][model] = f'val_losses_{model}.csv'
    best_model_files[model] = f'best_val_model_{model}_pytorch'

In [ ]:
plt.figure(figsize=(12, 8))
for model in model_names:
    # Check if the file exists
    if os.path.isfile(loss_files['train'][model]):
        # Load training loss
        df_train = pd.read_csv(loss_files['train'][model])
        epochs = df_train['Epoch']
        train_loss = df_train['Train_Loss']
        
        # Plot training loss curve
        plt.plot(epochs, train_loss, label=model)
    else:
        print(f"Training loss file {loss_files['train'][model]} does not exist.")

plt.title('Training Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for model in model_names:
    # Check if the file exists
    if os.path.isfile(loss_files['val'][model]):
        # Load validation loss
        df_val = pd.read_csv(loss_files['val'][model])
        epochs = df_val['Epoch']
        val_loss = df_val['Val_Loss']
        
        # Plot validation loss curve
        plt.plot(epochs, val_loss, label=model)
    else:
        print(f"Validation loss file {loss_files['val'][model]} does not exist.")

plt.title('Validation Loss Comparison')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
classification_reports = {}
accuracies = {}

for model in model_names:
    model_path = best_model_files[model]
    
    if os.path.isfile(model_path):
        model_loaded = torch.load(model_path)
        model_loaded.to(device)
        model_loaded.eval()
        
        print(f"\nEvaluating model: {model}")
        
        n_correct = 0
        n_total = 0
        all_targets = []
        all_predictions = []
        
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
                
                outputs = model_loaded(inputs)
                _, predictions = torch.max(outputs, 1)
                
                n_correct += (predictions == targets).sum().item()
                n_total += targets.shape[0]
                
                all_targets.append(targets.cpu().numpy())
                all_predictions.append(predictions.cpu().numpy())
        
        test_acc = n_correct / n_total
        accuracies[model] = test_acc
        print(f"Test Accuracy: {test_acc:.4f}")
        
        all_targets = np.concatenate(all_targets)
        all_predictions = np.concatenate(all_predictions)
        
        report = classification_report(all_targets, all_predictions, digits=4, output_dict=True)
        classification_reports[model] = report
    else:
        print(f"Best model file {model_path} does not exist.")

# Merge and Display Classification Reports
table_data = []

for model in model_names:
    if model in classification_reports:
        report = classification_reports[model]
        accuracy = accuracies[model]
        
        precision_0 = report['0']['precision']
        recall_0 = report['0']['recall']
        f1_score_0 = report['0']['f1-score']
        
        precision_1 = report['1']['precision']
        recall_1 = report['1']['recall']
        f1_score_1 = report['1']['f1-score']
        
        precision_2 = report['2']['precision']
        recall_2 = report['2']['recall']
        f1_score_2 = report['2']['f1-score']
        
        macro_avg_precision = report['macro avg']['precision']
        macro_avg_recall = report['macro avg']['recall']
        macro_avg_f1 = report['macro avg']['f1-score']
        
        weighted_avg_precision = report['weighted avg']['precision']
        weighted_avg_recall = report['weighted avg']['recall']
        weighted_avg_f1 = report['weighted avg']['f1-score']
        
        table_data.append([
            model,
            f"{precision_0:.4f}", f"{recall_0:.4f}", f"{f1_score_0:.4f}",
            f"{precision_1:.4f}", f"{recall_1:.4f}", f"{f1_score_1:.4f}",
            f"{precision_2:.4f}", f"{recall_2:.4f}", f"{f1_score_2:.4f}",
            f"{macro_avg_precision:.4f}", f"{macro_avg_recall:.4f}", f"{macro_avg_f1:.4f}",
            f"{weighted_avg_precision:.4f}", f"{weighted_avg_recall:.4f}", f"{weighted_avg_f1:.4f}",
            f"{accuracy:.4f}"
        ])

headers = [
    "Model",
    "Precision_0", "Recall_0", "F1-Score_0",
    "Precision_1", "Recall_1", "F1-Score_1",
    "Precision_2", "Recall_2", "F1-Score_2",
    "Macro Precision", "Macro Recall", "Macro F1-Score",
    "Weighted Precision", "Weighted Recall", "Weighted F1-Score",
    "Accuracy"
]

df_report = pd.DataFrame(table_data, columns=headers)
print(tabulate(df_report, headers='keys', tablefmt='pretty', showindex=False))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, model_name, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix for {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

# Define class names (adjust if necessary)
class_names = ['Class 0', 'Class 1', 'Class 2']

# Plot confusion matrices for each model
for model in model_names:
    if model in classification_reports:
        report = classification_reports[model]
        # Reconstruct y_true and y_pred from the classification report
        # Alternatively, store them during evaluation
        # Assuming you have stored all_targets and all_predictions
        # Here, we'll re-evaluate to get them again
        model_path = best_model_files[model]
        if os.path.isfile(model_path):
            model_loaded = torch.load(model_path)
            model_loaded.to(device)
            model_loaded.eval()
            
            all_targets = []
            all_predictions = []
            
            with torch.no_grad():
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)
                    
                    outputs = model_loaded(inputs)
                    _, predictions = torch.max(outputs, 1)
                    
                    all_targets.append(targets.cpu().numpy())
                    all_predictions.append(predictions.cpu().numpy())
            
            all_targets = np.concatenate(all_targets)
            all_predictions = np.concatenate(all_predictions)
            
            plot_confusion_matrix(all_targets, all_predictions, model, class_names)
    else:
        print(f"Cannot plot confusion matrix for {model} as it is missing in classification_reports.")